In [1]:
import pandas as pd
import requests
import json
import numpy as np
import reverse_geocoder as rg
import csv
import os

In [2]:
allcrime_path = "Resources/Crime_Data_from_2010_to_Present.csv"
allcrime = pd.read_csv(allcrime_path)
allcrime.dtypes

DR Number                   int64
Date Reported              object
Date Occurred              object
Time Occurred               int64
Area ID                     int64
Area Name                  object
Reporting District          int64
Crime Code                  int64
Crime Code Description     object
MO Codes                   object
Victim Age                  int64
Victim Sex                 object
Victim Descent             object
Premise Code              float64
Premise Description        object
Weapon Used Code          float64
Weapon Description         object
Status Code                object
Status Description         object
Crime Code 1              float64
Crime Code 2              float64
Crime Code 3              float64
Crime Code 4              float64
Address                    object
Cross Street               object
Location                   object
dtype: object

In [3]:
allcrime

,DR Number,Date Reported,Date Occurred,Time Occurred,Area ID,Area Name,Reporting District,Crime Code,Crime Code Description,MO Codes,...,Weapon Description,Status Code,Status Description,Crime Code 1,Crime Code 2,Crime Code 3,Crime Code 4,Address,Cross Street,Location
0,11401303,9/13/2010,9/12/2010,45,14,Pacific,1485,740,"VANDALISM - FELONY ($400 & OVER, ALL CHURCH VA...",329,...,NaN,IC,Invest Cont,740.0,NaN,NaN,NaN,SEPULVEDA BL,MANCHESTER AV,"(33.9599, -118.3962)"
1,70309629,8/9/2010,8/9/2010,1515,13,Newton,1324,946,OTHER MISCELLANEOUS CRIME,344,...,NaN,IC,Invest Cont,946.0,NaN,NaN,NaN,1300 E 21ST ST,NaN,"(34.0224, -118.2524)"
2,100100508,1/8/2010,1/7/2010,2005,1,Central,182,330,BURGLARY FROM VEHICLE,344,...,NaN,IC,Invest Cont,330.0,NaN,NaN,NaN,PICO BL,GRAND AV,"(34.0389, -118.2643)"
3,100100511,1/9/2010,1/6/2010,2100,1,Central,132,341,"THEFT-GRAND ($950.01 & OVER)EXCPT,GUNS,FOWL,LI...",0344 1402,...,NaN,IC,Invest Cont,341.0,998.0,NaN,NaN,200 S OLIVE ST,NaN,"(34.0538, -118.2488)"
4,100100523,1/15/2010,1/15/2010,245,1,Central,182,740,"VANDALISM - FELONY ($400 & OVER, ALL CHURCH VA...",329,...,NaN,AA,Adult Arrest,740.0,NaN,NaN,NaN,1100 S OLIVE ST,NaN,"(34.0409, -118.2609)"
5,100100552,1/23/2010,1/23/2010,1225,1,Central,192,237,CHILD NEGLECT (SEE 300 W.I.C.),1251,...,NaN,IC,Invest Cont,237.0,NaN,NaN,NaN,1300 S FLOWER ST,NaN,"(34.0401, -118.2668)"
6,100100562,1/28/2010,1/28/2010,2345,1,Central,152,626,INTIMATE PARTNER - SIMPLE ASSAULT,0416 2000,...,"STRONG-ARM (HANDS, FIST, FEET OR BODILY FORCE)",IC,Invest Cont,626.0,NaN,NaN,NaN,500 S GRAND AV,NaN,"(34.0502, -118.254)"
7,100100575,2/1/2010,1/31/2010,1720,1,Central,181,341,"THEFT-GRAND ($950.01 & OVER)EXCPT,GUNS,FOWL,LI...",1402,...,NaN,AA,Adult Arrest,341.0,NaN,NaN,NaN,1100 S FIGUEROA ST,NaN,"(34.0437, -118.2652)"
8,100100610,2/11/2010,2/10/2010,1800,1,Central,162,442,SHOPLIFTING - PETTY THEFT ($950 & UNDER),0325 1402,...,NaN,AA,Adult Arrest,442.0,998.0,NaN,NaN,700 W 7TH ST,NaN,"(34.048, -118.2577)"
9,100100616,2/11/2010,2/11/2010,1515,1,Central,192,341,"THEFT-GRAND ($950.01 & OVER)EXCPT,GUNS,FOWL,LI...",1212,...,NaN,AA,Adult Arrest,341.0,NaN,NaN,NaN,PICO,GRAND,"(34.0389, -118.2643)"


In [4]:
allcrime.drop(columns=['Date Reported','Time Occurred', 'Premise Description', 'Status Code', 'Crime Code 4',
                           'Address', 'Cross Street','DR Number','Reporting District', 'MO Codes', 'Crime Code 1', 'Crime Code 2',
                           'Crime Code 3'],inplace=True)

In [5]:
allcrime.dtypes

Date Occurred              object
Area ID                     int64
Area Name                  object
Crime Code                  int64
Crime Code Description     object
Victim Age                  int64
Victim Sex                 object
Victim Descent             object
Premise Code              float64
Weapon Used Code          float64
Weapon Description         object
Status Description         object
Location                   object
dtype: object

In [6]:
allcrime["Date Occurred"]=pd.to_datetime(allcrime["Date Occurred"])

In [7]:
allcrime['Year Occurred'] = allcrime['Date Occurred'].apply(lambda yr:yr.year)

In [8]:
allcrime['Year Occurred'].value_counts()

2017    229898
2018    226807
2016    224625
2015    214810
2019     88584
2010     63851
Name: Year Occurred, dtype: int64

In [9]:
allcrime

,Date Occurred,Area ID,Area Name,Crime Code,Crime Code Description,Victim Age,Victim Sex,Victim Descent,Premise Code,Weapon Used Code,Weapon Description,Status Description,Location,Year Occurred
0,2010-09-12,14,Pacific,740,"VANDALISM - FELONY ($400 & OVER, ALL CHURCH VA...",0,M,W,101.0,NaN,NaN,Invest Cont,"(33.9599, -118.3962)",2010
1,2010-08-09,13,Newton,946,OTHER MISCELLANEOUS CRIME,0,M,H,103.0,NaN,NaN,Invest Cont,"(34.0224, -118.2524)",2010
2,2010-01-07,1,Central,330,BURGLARY FROM VEHICLE,46,M,H,101.0,NaN,NaN,Invest Cont,"(34.0389, -118.2643)",2010
3,2010-01-06,1,Central,341,"THEFT-GRAND ($950.01 & OVER)EXCPT,GUNS,FOWL,LI...",55,M,W,710.0,NaN,NaN,Invest Cont,"(34.0538, -118.2488)",2010
4,2010-01-15,1,Central,740,"VANDALISM - FELONY ($400 & OVER, ALL CHURCH VA...",24,F,W,102.0,NaN,NaN,Adult Arrest,"(34.0409, -118.2609)",2010
5,2010-01-23,1,Central,237,CHILD NEGLECT (SEE 300 W.I.C.),11,M,H,502.0,NaN,NaN,Invest Cont,"(34.0401, -118.2668)",2010
6,2010-01-28,1,Central,626,INTIMATE PARTNER - SIMPLE ASSAULT,28,M,O,108.0,400.0,"STRONG-ARM (HANDS, FIST, FEET OR BODILY FORCE)",Invest Cont,"(34.0502, -118.254)",2010
7,2010-01-31,1,Central,341,"THEFT-GRAND ($950.01 & OVER)EXCPT,GUNS,FOWL,LI...",36,M,W,710.0,NaN,NaN,Adult Arrest,"(34.0437, -118.2652)",2010
8,2010-02-10,1,Central,442,SHOPLIFTING - PETTY THEFT ($950 & UNDER),23,M,B,404.0,NaN,NaN,Adult Arrest,"(34.048, -118.2577)",2010
9,2010-02-11,1,Central,341,"THEFT-GRAND ($950.01 & OVER)EXCPT,GUNS,FOWL,LI...",0,M,H,102.0,NaN,NaN,Adult Arrest,"(34.0389, -118.2643)",2010


In [10]:
indexNames = allcrime[allcrime['Year Occurred'] == 2010 ].index
 
allcrime.drop(indexNames , inplace=True)

allcrime

indexNames2 = allcrime[allcrime['Year Occurred'] == 2011 ].index
 
allcrime.drop(indexNames2 , inplace=True)

indexNames3 = allcrime[allcrime['Year Occurred'] == 2012 ].index
 
allcrime.drop(indexNames3 , inplace=True)

indexNames4 = allcrime[allcrime['Year Occurred'] == 2013 ].index
 
allcrime.drop(indexNames4 , inplace=True)

indexNames5 = allcrime[allcrime['Year Occurred'] == 2014 ].index
 
allcrime.drop(indexNames5 , inplace=True)

indexNames6 = allcrime[allcrime['Year Occurred'] == 2015 ].index
 
allcrime.drop(indexNames6 , inplace=True)

indexNames7 = allcrime[allcrime['Year Occurred'] == 2019 ].index
 
allcrime.drop(indexNames7 , inplace=True)


In [11]:
allcrime['Year Occurred'].value_counts()

2017    229898
2018    226807
2016    224625
Name: Year Occurred, dtype: int64

In [12]:
allcrime

,Date Occurred,Area ID,Area Name,Crime Code,Crime Code Description,Victim Age,Victim Sex,Victim Descent,Premise Code,Weapon Used Code,Weapon Description,Status Description,Location,Year Occurred
245147,2016-07-13,5,Harbor,310,BURGLARY,34,F,H,501.0,NaN,NaN,Invest Cont,"(33.8028, -118.2623)",2016
246300,2016-07-13,5,Harbor,442,SHOPLIFTING - PETTY THEFT ($950 & UNDER),0,X,X,403.0,NaN,NaN,Adult Other,"(33.7397, -118.2923)",2016
246765,2016-07-13,5,Harbor,740,"VANDALISM - FELONY ($400 & OVER, ALL CHURCH VA...",82,F,W,501.0,NaN,NaN,Invest Cont,"(33.7279, -118.2836)",2016
247008,2016-07-14,5,Harbor,510,VEHICLE - STOLEN,0,NaN,NaN,101.0,NaN,NaN,Adult Arrest,"(33.7298, -118.3056)",2016
247028,2016-07-13,5,Harbor,510,VEHICLE - STOLEN,0,NaN,NaN,101.0,NaN,NaN,Invest Cont,"(33.8315, -118.3065)",2016
247696,2016-07-05,5,Harbor,440,THEFT PLAIN - PETTY ($950 & UNDER),39,M,A,710.0,NaN,NaN,Invest Cont,"(33.7791, -118.2415)",2016
248198,2016-06-09,5,Harbor,440,THEFT PLAIN - PETTY ($950 & UNDER),74,M,W,406.0,NaN,NaN,Invest Cont,"(33.7941, -118.29)",2016
259469,2016-07-13,5,Harbor,626,INTIMATE PARTNER - SIMPLE ASSAULT,36,M,B,501.0,400.0,"STRONG-ARM (HANDS, FIST, FEET OR BODILY FORCE)",Invest Cont,"(33.7406, -118.2945)",2016
259978,2016-07-02,5,Harbor,441,THEFT PLAIN - ATTEMPT,55,M,O,203.0,NaN,NaN,Adult Other,"(33.8583, -118.2971)",2016
260157,2016-07-14,5,Harbor,900,VIOLATION OF COURT ORDER,43,M,W,502.0,NaN,NaN,Adult Other,"(33.7182, -118.2949)",2016


In [13]:
allcrime['Year Occurred'].value_counts()

2017    229898
2018    226807
2016    224625
Name: Year Occurred, dtype: int64

In [14]:
allcrime.to_csv(r'C:\Users\josep\Project1\Resources\Crime_Data_Cleanup.csv')